In [2]:
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import scipy.sparse
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight

In [3]:
data = "C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\Data\\scGNN_matrix.h5ad"

In [4]:
adata = sc.read_h5ad(data)
print('X matrix is sparse:', scipy.sparse.issparse(adata.X))
print('X size =', adata.X.shape)

X matrix is sparse: False
X size = (2000, 8013)


In [5]:
adata

AnnData object with n_obs × n_vars = 2000 × 8013
    obs: 'my_clust_1'

In [6]:
print(adata.X)
print(adata.X.shape)
print(type(adata.X))
print(adata.X.dtype)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.23346782 0.01299769 0.08607478 ... 0.08972513 0.         0.2212397 ]
 [0.2330591  0.         0.00645433 ... 0.         0.         0.22836858]
 [0.29366922 0.         0.05959742 ... 0.1592918  0.01173312 0.2759499 ]]
(2000, 8013)
<class 'numpy.ndarray'>
float32


In [7]:
cluster_type = 'my_clust_1'

In [8]:
adata.obs[cluster_type]

TGCGTGGCATGACATC_sc69_1     5
CAACCTCAGCTCTCGG_sc69_1     5
TCGTACCCATCCCATC_sc72_1     5
GTACTTTTCTAACGGT_sc69_1     5
CCTAAAGCATAACCTG_sc72_1     5
                           ..
TTCTTAGTCTCGAGTA_sc72_1     5
CATATGGCACACCGCA_sc69_1    17
CTCCTAGGTTGGTAAA_sc69_1    17
GGGTCTGAGGGCATGT_sc69_1    17
CCTTCCCCACCTCGTT_sc69_1    17
Name: my_clust_1, Length: 2000, dtype: category
Categories (30, object): ['0', '10', '11', '11b', ..., '7c', '8', '9', '9b']

In [9]:
annotation_dict= {
    '9': 'CAP1',
    '24': 'CAP2',
    '9b': 'VEC',
    '27': 'LEC',
        
    '17': 'Ciliated',
    '15': 'Secretory',
    '22': 'AT1',
    '6': 'AT2',
    '12': 'AT2-t1',
    '19': 'AT2-t2',
        
    '14': 'AF',
    '25': 'Pericyte',
        
    '20': 'Mesothelial',
        
    '3': 'B1',
    '3b': 'B2',
        
    '0': 'Th1',
    '8': 'Tnaive',
    '11': 'Tex',
    '77': 'Treg',
        
    '11b': 'NK',
        
    '4a': 'AM',
    '4': 'M-t1',
    '10': 'M-lc',
    '7': 'M-t2',
    '7b': 'M-C1q',
    '7c': 'iMon',
        
    '23': 'pDC',
    '13': 'DC',
    '5b': 'N1',
    '5': 'N2',
}

In [10]:
adata.obs['cell_type_edit'] = [annotation_dict[clust] for clust in adata.obs[cluster_type]]

dict_list = list(annotation_dict.keys())
adata_list = list(adata.obs[cluster_type].unique())
print('Keys in dictionary not in adata:', [item for item in dict_list if item not in adata_list] )
print('Keys in adata not in dictionary:', [item for item in adata_list if item not in dict_list] )

Keys in dictionary not in adata: ['24', '27', '12', '19', '3', '3b', '8', '11', '77', '11b', '4a', '4', '7b', '23', '13']
Keys in adata not in dictionary: []


In [11]:
adata.obs['cell_type_edit']

TGCGTGGCATGACATC_sc69_1          N2
CAACCTCAGCTCTCGG_sc69_1          N2
TCGTACCCATCCCATC_sc72_1          N2
GTACTTTTCTAACGGT_sc69_1          N2
CCTAAAGCATAACCTG_sc72_1          N2
                             ...   
TTCTTAGTCTCGAGTA_sc72_1          N2
CATATGGCACACCGCA_sc69_1    Ciliated
CTCCTAGGTTGGTAAA_sc69_1    Ciliated
GGGTCTGAGGGCATGT_sc69_1    Ciliated
CCTTCCCCACCTCGTT_sc69_1    Ciliated
Name: cell_type_edit, Length: 2000, dtype: object

In [12]:
unique_celltype_sub = adata.obs['cell_type_edit'].unique()
print(unique_celltype_sub)

num_unique_celltype_sub = adata.obs['cell_type_edit'].nunique()
print(f"Number of unique sub cell types: {num_unique_celltype_sub}")

['N2' 'M-lc' 'N1' 'Mesothelial' 'iMon' 'Ciliated' 'CAP1' 'M-t2' 'Th1'
 'Secretory' 'AF' 'VEC' 'Pericyte' 'AT1' 'AT2']
Number of unique sub cell types: 15


In [ ]:
replacement_dict = {
    'N2': 0,
    'M-lc': 1,
    'N1': 2,
    'Mesothelial': 3,
    'iMon': 4,
    'Ciliated': 5,
    'CAP1': 6,
    'M-t2': 7,
    'Th1': 8,
    'Secretory': 9,
    'AF': 10,
    'VEC': 11,
    'Pericyte': 12,
    'AT1': 13,
    'AT2': 14
}